In [1]:
import os
import sys

# Add project root to path to allow importing from model
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from model.utils.validation_utils import test_prop_partial_discretization_matches_ode, plot_partial_matches_ode_with_cliff, plot_forward_euler_vs_ode


# To make plots appear in the notebook
%matplotlib inline

# Model Validations

The purpose of this notebook is to validate the outputs generated by our discretized, modified Lotka-Volterra (LV) agent-based model (ABM) as being consistent with expectations given the equations of the system and the goals of our implementation.

## Introduction To The System

The standard LV system is described as:

$$
\begin{align}
\frac{ds}{dt} &= \alpha x - \beta xy \\
\frac{dw}{dt} &= -\gamma y + \delta xy
\end{align}
$$

where (adapted from [Wikipedia's Lotka-Volterra equations page](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations)):[^1]

- The variable $x$ is the population of prey (for example, the number of rabbits per square kilometre)
- The variable $y$ is the population of some predator (for example, the number of foxes per square kilometre)  
- $\frac{dy}{dt}$ and $\frac{dx}{dt}$ represent the instantaneous growth rates of the two populations
- $t$ represents time
- The prey's parameters, $\alpha$ and $\beta$, describe, respectively, the maximum prey per capita growth rate, and the effect of the presence of predators on the prey death rate
- The predator's parameters, $\gamma$ and $\delta$, respectively describe the predator's per capita death rate, and the effect of the presence of prey on the predator's growth rate
- All parameters are positive and real

### Introduction

Our implementation of Lotka-Volterra employs wolves (predators) and sheep (prey) as model organisms. Additionally, our system employs an additional parameter in complex with the intrinsic $\beta$ parameter of "base" LV called $\theta$. The complex is added to both equations of the system in order to add and balance a "predation intensity" term which is somewhat like the one proposed by Holling (1973). However, for the purposes of our validation, the $\theta\ parameter will always be set to one, thus "washing out" of our model. So:

$$
\begin{align}
\frac{ds}{dt} &= \alpha s - \theta\beta sw \\
\frac{dw}{dt} &= -\gamma w + \theta\beta\delta sw
\end{align}
$$

where $w$ is number of wolves and $s$ number of sheep. Elsewhere in this notebook we will omit the $\theta$ variable.

In our "partially discretized" model implementation of the LV system, the continuous curve of the predator population, wolves, is replaced by the use of a model implementation in which wolf "agents" are individualized and have discrete, integer population counts at every step. The discrete population notably cannot rebound from a sub-1 value.  The prey population, the sheep, are left to operate as a single population group, with non-discrete (implemented as a floating point number, so our discretion is limited!) populations that can exist at sub-1 counts.

The implementation of discretized systems is widely discussed and debated in simulation and agent-based modeling literature. [CITATIONS]

Our approach is based on the forward Euler method for solving ordinary differential equations (ODE). The details of our approach are described in the accompanying notebook, `introducing_do_android_wolves.ipynb`. To simplify, each step [DETAILS] 

*** [Solvers details] ***

## Hypotheses

Hypothesis One

We can first hypothesize that an idiomatic Python implementation of a forward Euler solution to (modified) LV will depart from the solution generated by a Solver with an error-per step ratio corresponding to known calculations [known calculations]

Hypothesis Two

Next, we can hypothesize that our partially discretized ABM implementation will depart from the solution generated by a Solver with a similar error-per step ratio.




## Test One: Forward-Euler versus solver


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate

# The reference function: modified LV.
def reference_function(x, t, alpha, beta, gamma, delta):  # noqa
    # Note the omission of theta
    s, w = x
    ds_dt = alpha * s - beta * s * w
    dw_dt = -gamma * w + beta * delta * s * w
    return [ds_dt, dw_dt]

def partial_reference_function(x, t, alpha, beta, gamma, delta):  # noqa
    # Note the omission of theta
    s, w = x
    ds_dt = alpha * s - beta * s * w
    dw_dt = -gamma * w + beta * delta * s * w
    return [ds_dt, dw_dt]


In [ ]:
def modified_lotkavolterra(t, x, a, b, c, d):
    s, w = x
    return [a*s - b*s*w, -c*w + b*d*s*w]


# Solve the ODE
def solve_mlv(x, max_step, a, b, c, d, method, teval=None):
    S = integrate.solve_ivp(
        modified_lotkavolterra,
        [0, max_step],
        x,
        args=(a, b, c, d),
        method=method,
        rtol=1.49012e-8,
        atol=1.49012e-8,
        t_eval=teval,
        dense_output=True
    )
    return S

alpha = 1.0
beta = 0.2
gamma = 0.3
delta = 0.1
x = [5, 5]
max_step = 200

solvers = ["RK45", "RK23", "DOP853", "LSODA", "BDF", "Radau"]

# default: RK45
S1 = solve_mlv(x, max_step, alpha, beta, gamma, delta, solvers[0])
S2 = solve_mlv(x, max_step, alpha, beta, gamma, delta, solvers[1])
S3 = solve_mlv(x, max_step, alpha, beta, gamma, delta, solvers[2])
S4 = solve_mlv(x, max_step, alpha, beta, gamma, delta, solvers[3])
S5 = solve_mlv(x, max_step, alpha, beta, gamma, delta, solvers[4])
S6 = solve_mlv(x, max_step, alpha, beta, gamma, delta, solvers[5])

solutions = [S1, S2, S3, S4, S5, S6]

for i, sol in enumerate(solutions):
    print(f"Solver: {solvers[i]}")
    print(f"Final sheep: {sol.y[0][-1]}")
    print(f"Final wolves: {sol.y[1][-1]}")
    print("\n")

print("Mean Results:")
print(f"Final sheep: {np.mean([sol.y[0][-1] for sol in solutions])}")
print(f"Final wolves: {np.mean([sol.y[1][-1] for sol in solutions])}")
print("\n")

# Plot using np.linspace from the scipy example
t = np.linspace(0, max_step, 2000)

fig, axes = plt.subplots(3, 2, figsize=(14, 18))
fig.suptitle('Comparison of ODE Solvers for Modified Lotka-Volterra System', fontsize=16)
axes = axes.flatten()

for i, sol in enumerate(solutions):
    z = sol.sol(t)
    ax = axes[i]
    ax.plot(t, z.T)
    ax.set_xlabel('Time (t)')
    ax.set_ylabel('Population')
    ax.legend(['sheep (s)', 'wolves (w)'], shadow=True)
    ax.set_title(f'Solver: {solvers[i]}')
    ax.grid(True)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:


def forward_euler_estimator(initial_conditions, dt, max_step, a, b, c, d):
    # Deal with time and initiailize the solution array
    t = np.arange(0, max_step + dt, dt)  # all the steps to max_step counting from 0
    # The referenced code works on one variable/one equation, here we simply need two state arrays.
    s = np.zeros(len(t))
    w = np.zeros(len(t))
    s[0] = initial_conditions[0]
    w[0] = initial_conditions[1]

    # since we have two functions in our system, we need to split them into their own lambda functions.
    f_s = lambda t, s, w: a * s - b * s * w             # noqa
    f_w = lambda t, s, w: -c * w + b * d * s * w        # noqa
    # note: autonomous functions, t is not actually used to process.

    # Here is the code from the referenced notebook. We need to do this twice, basically
    # for i in range(0, len(t) - 1):
    # s[i + 1] = s[i] + h*f(t[i], s[i])
    # note that h in this example is dt for us.

    for i in range(0, len(t) - 1):
        s[i + 1] = s[i] + dt*f_s(t[i], s[i], w[i])
        w[i + 1] = w[i] + dt*f_w(t[i], s[i], w[i])

    return t, s, w

# Model parameters for oscillatory behavior
alpha = 1.0
beta = 0.2
gamma = 0.3
delta = 0.1
s_start = 5
w_start = 5
dt = 0.02
max_step = 200

x = [s_start, w_start]

# Time array for the solvers
t = np.arange(0, max_step + dt, dt)

# approximate solution from forward euler
t_euler, s_euler, w_euler = forward_euler_estimator(x, dt, max_step, alpha, beta, gamma, delta)

# 2. "exact" solution from solver, above
# we can try plugging in different solvers here if we like, though generally the outcomes are the same
S = solve_mlv(x, max_step, alpha, beta, gamma, delta, "RK45")

# align points with time
timed_S = S.sol(t)
s_exact = timed_S[0]
w_exact = timed_S[1]

plot_forward_euler_vs_ode(t, s_euler, w_euler, s_exact, w_exact)  # call to our external plotting function




In [ ]:
# Use integer values for wolves
# Use solver outputs for sheep.
max_step = 200
def partially_discrete(initial_conditions, dt, max_step, a, b, c, d):
    # Deal with time and initiailize the solution array
    t = np.arange(0, max_step + dt, dt)  # all the steps to max_step counting from 0
    # The referenced code works on one variable/one equation, here we simply need two state arrays.
    s = np.zeros(len(t))
    w = np.zeros(len(t))
    s[0] = initial_conditions[0]
    w[0] = initial_conditions[1]

    # since we have two functions in our system, we need to split them into their own lambda functions.
    # f_s = lambda t, s, w: a * s - b * s * w             # not needed here!
    f_w = lambda t, s, w: -c * w + b * d * s * w        # noqa
    # note: autonomous functions, t is not actually used to process.

    # Here is the code from the referenced notebook. We need to do this twice, basically
    # for i in range(0, len(t) - 1):
    # s[i + 1] = s[i] + h*f(t[i], s[i])
    # note that h in this example is dt for us.

    accumulator = 0.0 # for partials
    for i in range(0, len(t) - 1):
        s_growth_rate = a - b * w[i]
        s[i+1] = s[i] * np.exp(s_growth_rate * dt)

        # Calculate the change in wolves (this will be fractional)
        dw = dt * f_w(t[i], s[i+1], w[i])
        
        # Add the change to the accumulator
        accumulator += dw
        
        # Extract the integer part and update the wolf population
        if accumulator >= 1.0:
            # Positive growth: take the floor
            integer_change = int(accumulator)
            w[i+1] = w[i] + integer_change
            accumulator -= integer_change
        elif accumulator <= -1.0:
            # Negative growth: take the ceiling (more negative)
            integer_change = int(accumulator)  # This will be negative
            w[i+1] = w[i] + integer_change
            accumulator -= integer_change
        else:
            # No integer change yet
            w[i+1] = w[i]
        
        # Ensure wolves don't go below 0
        if w[i+1] < 0:
            w[i+1] = 0
            accumulator = 0.0  # Reset accumulator when extinct

        # Debug print to see what's happening
        # print(f"s[{i+1}]: {s[i+1]:.3f}, w[{i+1}]: {w[i+1]}, dw: {dw:.3f}, acc: {accumulator:.3f}")

    return t, s, w

t_p, s_p, w_p = partially_discrete(x, dt, max_step, alpha, beta, gamma, delta)

plot_forward_euler_vs_ode(t_p, s_p, w_p, s_exact, w_exact)  # call to our external plotting function


## Data assembly

We will be performing several runs through the LV system in this notebook, and some parameters will vary from run to run. Generally speaking for the project we are not so interested in the outcome permutations generated by the traditional LV variables, as the effects of different settings of these parameters is well-settled science. However, we might be interested to confirm whether or not our system diverges from the main system, so we will investigate this in a bit.

To get started, we will use the parameters we employ throughout the study:
```
        "alpha": 1,
        "beta": 0.1,
        "gamma": 1.5,
        "delta": 0.75,
```

This means that our system equations will evaluate as:

```
        ds_dt = 1 * s - 0.1 * s * w
        dw_dt = -1.5 * w + 0.75 * s * w
```

... where `w` and `s` are our sheep and wolves counts. Let's take a look at the solver working with these equations, 


and we will watch the ODE solver compute an output using these numbers.

## A note on time and solvers

One theoretical difference between our discretized model and the continous functions they are intended to instantiate is the modeling application of finite time. Effectively, in order for our model to be a model, our model agents need discrete time points at which they can observe and interact with the environment. Meanwhile, since we are attempting to orchestrate domain processes that operate according to the continous system, our discrete time points also need enough organization so that we can reassemble their history in a way that they can be compared against a similar amount of continuous time. We accomplish this, in the case of this and many other models, using "steps," which are points in time spaced equally using a variable we call `dt`. This variable name is of course not an accident; our `dt` is designed to match up with the same "delta time (dt)" variable governing the continuous system.

As a result of our use of discrete time points, it is expected that our simulation will operate in a more "coarse" fashion than the underlying continuous system. However, another concept to note here is that we will use a "continuous equation solver" (`scipy-integrate.solve-ivp`)implemented for Python to represent the output of the continuous equations. That program, being a computer program running in our universe, also uses discrete steps, although by default they are far finer than the ones that we use in our model. Nonetheless, it should be acknowledged that we are working with discrete computing with both systems, and that we might expect for the output solutions to conform more closely as the gap between time-step scales of the two systems is reduced (for instance, as `dt` is set smaller in our discretized version.)

In [ ]:
model_params = {
        "alpha": 1.0,
        "beta": 1.0,
        "gamma": 1.0,
        "delta": 1.0,
        "s_start": 20,
        "w_start": 5,
        "dt": 0.02,
        "steps": 200,
        "sheep_max": 210,
        "eps": 0.0
    }

ode_r, ode_no_cliff_r, partial_r = test_prop_partial_discretization_matches_ode(model_params)


In [ ]:
plot_partial_matches_ode_with_cliff(model_params, partial_r, ode_r, ode_no_cliff_r)

In [ ]:
model_params["alpha"] = 1.1
model_params["beta"] = 0.4
model_params["gamma"] = 0.4
model_params["delta"] = 0.1

ode_r, ode_no_cliff_r, partial_r = test_prop_partial_discretization_matches_ode(model_params)


In [ ]:
plot_partial_matches_ode_with_cliff(model_params, partial_r, ode_r, ode_no_cliff_r)

In [ ]:
model_params["alpha"] = 1.0
model_params["beta"] = 0.2
model_params["gamma"] = 0.3
model_params["delta"] = 0.1

model_params["steps"] = 20000

ode_r, ode_no_cliff_r, partial_r = test_prop_partial_discretization_matches_ode(model_params)


In [ ]:
plot_partial_matches_ode_with_cliff(model_params, partial_r, ode_r, ode_no_cliff_r)

### Notes

[^1] The Wikipedia page uses the word, "density," to describe the function of the varibles x and y. We think this is both incorrect and specifically a departure from the works of Lotka and Volterra. We have removed the word from our description.